In [9]:
TIME_DATA_JSON = 'time_data.json'

In [10]:
# 读取配置
import json

def read_json_file(file_name: str):
    with open(file_name, 'r', encoding="utf-8") as file:
        data = json.load(file)
    return data

def write_dict_to_json(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

coze_config = read_json_file('config-coze.json')
print(coze_config)

{'space_id': '7295668070847102994', 'bot_id': '7347955889028202514', 'api_key': 'u7yf5zGfvyQiYoIR5kdxNi4C55s2ov4BFQUgAax7Uhtmz8vcAkrMwL6jbtIeCpGq'}


In [11]:
import requests
import json

def post_to_chat_API(query):
    headers = {
        'Authorization': f'Bearer {coze_config["api_key"]}',
        'Content-Type': 'application/json',
        'Accept': '*/*',
        'Host': 'api.coze.com',
        'Connection': 'keep-alive'
    }

    data = {
        "bot_id": f"{coze_config['bot_id']}",
        "user": "1145141919810",
        "query": query,
        "stream": False
    }

    response = requests.post('https://api.coze.com/open_api/v2/chat', headers=headers, data=json.dumps(data))
    
    return response

In [12]:
import time

res_dict = read_json_file(TIME_DATA_JSON)
for i in range(0,3):   # retry
    for hour in range(0,24):
        for minute in range(0,60):
            if minute < 10: 
                minute = "0" + str(minute)
            time_str = f"{hour}:{minute}"
            if time_str in res_dict.keys():
                continue

            print(time_str)
            try:
                LLM_response = json.loads(post_to_chat_API(time_str).content)
                res_dict[time_str] = LLM_response['messages'][0]['content']
                print(res_dict[time_str])
                write_dict_to_json(TIME_DATA_JSON, res_dict)
            except Exception as e:
                print(e)
            time.sleep(3)
